# FPGA Workshop 2 : Building an SoC
Objectives:
* Building an SoC
* Creating custom peripherals
* building and running and software on our SoC

# Last Episode...

We learnt what an FPGA is
<img src="files/files/ecp5_block_diagram.png">

We learnt how to write combinatory and synchronous logic in migen

In [1]:
from migen import *

class mymodule(Module):
    def __init__(self, a, b, difference):
        # a and b must be Signal() instances
        y = Signal()
        y_ff = Signal()
        self.comb += [ # combinatorial: 'instantaneous'
            y.eq(a + b),
            difference.eq(a-b)
        ]
        self.sync += [ # synchronous: y_ff value is updated on next 'sys_clk' rising edge
            y_ff.eq(a+b)
        ]
        useless = Signal() # not used anywhere, won't generate logic

        self.y = y # will be acessible by parent Module
        # y_ff is not accessible by parent Module, thus, can't be connected to IOs or other modules.
        # it will be removed with its associated logic during the optimization phase

We learnt how to describe our system using litex's `Platform` and expand it

In [2]:
# the Platform represents the ulx3s 'capabilities', which FPGA brand/type is uses etc
from litex_boards.platforms.ulx3s import Platform
from litex.build.generic_platform import * # so we can use Pins(), etc to extend the platform

def buildplat():
    # by default, LiteX uses Lattice's closed-source toolchain.
    # ULX3S has 4 FPGA size variants, set 'device' accordingly
    plat = Platform(toolchain="trellis", device="LFE5U-12F")

    # these IOs should probably be defined by default in litex, but since it's not we have to do it
    plat.add_extension([
        ("user_button", 0, Pins("D6"), IOStandard("LVCMOS33")),  # BTN_PWRn (inverted logic)
        ("user_button", 1, Pins("R1"), IOStandard("LVCMOS33")),  # FIRE1
        ("user_button", 2, Pins("T1"), IOStandard("LVCMOS33")),  # FIRE2
        ("user_button", 3, Pins("R18"), IOStandard("LVCMOS33")), # UP
        ("user_button", 4, Pins("V1"), IOStandard("LVCMOS33")),  # DOWN
        ("user_button", 5, Pins("U1"), IOStandard("LVCMOS33")),  # LEFT
        ("user_button", 6, Pins("H16"), IOStandard("LVCMOS33")), # RIGHT
    ])
    return plat

Then how to request pins, and build the system and load it onto the ULX3s

In [3]:
# A bit of boilerplate...
from migen import * # the FHDL
from litex import * # the SoC builder

# DEMO 1: LED display state of push buttons
class hello(Module):
    def __init__(self, plat):
        self.platform = plat
        self.ios = set()
        for i in range(7):
            led = plat.request("user_led", i)
            btn = plat.request("user_button", i)
            self.comb += led.eq(btn)
            self.ios = self.ios | {led, btn}

plat = buildplat()
b = plat.build(hello(plat), run=True)
prog = plat.create_programmer().load_bitstream('build/top.bit')

# Now, let's make a SoC!

System on Chip = CPU + periperals (IOs, accelerators) + memory

<img src="files/files/precursor-soc-diagram.png">
https://www.crowdsupply.com/sutajio-kosagi/precursor/updates/a-guided-tour-of-the-precursor-system-on-chip-soc

# CPU
Litex gives a couple of CPUs to try from: vexrisc, lm32, minerva, mor1kx (and a lot more). Most of them use the Risc V ISA but you also find OPEN POWER, OpenRISC or other odball ISA.
All of them have a "wrapper" which typically contain a CPU-to-wishbone-interface, reset&interrupt signals.

We'll be using `vexriscv` CPU, as it's a good performance / size compromize and well supported by other projects like https://github.com/litex-hub/linux-on-litex-vexriscv

Its peripherals and memories will be connected through a Wishbone bus, which is a rather simple bus well suited to FPGAs SoC where typically only a few peripherals are connected.

# Memory
The ULX3s has a 32MB SDRAM onboard, which can be hooked to the Wishbone bus through a `litedram` core.

litex also creates a small ROM and RAM (using FPGA block RAM), which we will need to run the bootloader.

# Peripherals
litex provides plenty of useful peripherals, from basic ones like GPIO, UART, SPI, I2C, PWM to more complex peripherals like Ethernet, Framebuffer, PCIe endpoint, USB

We'll start with the basics:
* an UART, so we can have a serial terminal
* a timer
* a LED driver

We will also create our own peripheral using CSRs to control the LEDs (let's be honest, blinking lights is all that Electronics is for :)

# Out of the box SoC
Fortunately, litex_boards provides a way to build a basic SoC on the ULX3s, without the need for writing any Python code. This is done in litex-boards package: https://github.com/litex-hub/litex-boards/blob/master/litex_boards/targets/ulx3s.py  
litex_board can be installed with this command: `pip3 install --user git+https://github.com/litex-hub/litex-boards.git@47faaf20d50e58affda81f611561efdc073c660d`

It can be invoqued as a command-line tool. Run the following command (don't forget to source the virtualenv if you used one):  
`python -m litex_boards.targets.ulx3s --help`

We can build a complete SoC with this command (don't forget to change the device accordingly to the board you have):  
`python -m litex_boards.targets.ulx3s --device LFE5U-85F --build`

## (optional) Generating documentation
Litex can generate the documentation for your SoC. This will require `sphinx` and `spinxconfrib-wavdrom`
```
pip install sphinx spinxconfrib-wavdrom
python -m litex_boards.targets.ulx3s --device LFE5U-85F --no-compile-software --doc
python3 -m http.server --directory build/ulx3s/doc/_build/html 8080
```
Now open http://localhost:8080/

# Playing with our SoC
1. Load the SoC : `python -m litex_boards.targets.ulx3s --no-compile-software --load`
2. open a terminal:  `lxterm /dev/ttyUSB0` or `ujprog -t -P /dev/ttyUSB0` or `picocom -b 115200 /dev/ttyUSB0`
3. Try reading and writing a couple of CSR  
   `mem_write 0xf0000000 0`  
   `mem_write 0xf0002000 0x23`  
   `leds 3`

# Building and running application on the SoC
Once again, litex has some useful examples. A port of "lab004" firmware from https://github.com/litex-hub/fpga_101 is located in `lab004_firmware` directory.

You can build and load it by running:  
`cd lab004_firmware && make && lxterm /dev/ttyUSB0 --kernel firmware.bin`

```
(env) max@LAPTOP-32ULRLMS:~/FPGA/old/fpga_workshop/workshop2$ lxterm /dev/ttyUSB4 --kernel lab004_firmware/firmware.bin

        __   _ __      _  __
       / /  (_) /____ | |/_/
      / /__/ / __/ -_)>  <
     /____/_/\__/\__/_/|_|
   Build your hardware, easily!

 (c) Copyright 2012-2020 Enjoy-Digital
 (c) Copyright 2007-2015 M-Labs

 BIOS built on Mar 25 2021 10:11:52
 BIOS CRC passed (21ce4340)

 Migen git sha1: --------
 LiteX git sha1: 0e7d8219

--=============== SoC ==================--
CPU:            VexRiscv @ 50MHz
BUS:            WISHBONE 32-bit @ 4GiB
CSR:            32-bit data
ROM:            32KiB
SRAM:           8KiB
L2:             8KiB
SDRAM:          32768KiB 16-bit @ 50MT/s (CL-2 CWL-2)

--========== Initialization ============--
Initializing SDRAM @0x40000000...
Switching SDRAM to software control.
Switching SDRAM to hardware control.
Memtest at 0x40000000 (2MiB)...
   Read: 0x40000000-0x40200000 2MiB
Memtest OK
  Write speed: 11MiB/s (2Mi
   Read speed: 10MiB/s

--============== Boot ==================--
Booting from serial...
Press Q or ESC to abort boot completely.
sL5DdSMmkekro
[LXTERM] Received firmware download request from the device.
[LXTERM] Uploading lab004_firmware/firmware.bin to 0x40000000 (8036 bytes)...
[LXTERM] Upload complete (9.8KB/s).
[LXTERM] Booting the device.
[LXTERM] Done.
Executing booted program at 0x40000000

--============= Liftoff! ===============--

Lab004 - CPU testing software built Mar 25 2021 12:05:41

Available commands:
help                            - this command
reboot                          - reboot CPU
led                             - led test
RUNTIME>
```

# Our own peripherals
Our ULX3s feels a bit under-used with its deaf buttons, so let's make a simple peripheral to read buttons states.

## Step 1: create our peripheral
We will be using CSR (Control Status Register) to expose the button status on memory-mapped registers

In [18]:
# This is our button peripheral
from migen import *
from litex.soc.interconnect.csr import *

class Buttons(Module, AutoCSR):
    # our Module inherits AutoCSR, which allows litex to automatically
    # gather this module's CSRs and map them
    def __init__(self, plat):
        button_names = ["UP", "DOWN", "LEFT", "RIGHT"]
        csrfields = [
            CSRField(
                name=name,
                size=1,
                description="Button {} state".format(name),
                values = [("0", "Button not pressed"), ("1", "Button pressed")]
            ) for name in button_names
        ]
        self.buttons = CSRStatus(
            fields = csrfields,
            description = "Buttons status"
        )
        # self.buttons.fields.UP.eq(plat.request("button_UP", 0))
        self.comb += [
            getattr(self.buttons.fields, name).eq(plat.request("button_"+name, 0))
            for i, name in enumerate(button_names)
        ]

## Step 2: make our custom SoC
To make a custom SoC, you typically make a derived class from `SoCCore`, and instanciate peripherals in it's constructor.
`litex_boards.platforms.ulx3s.Platform` is actually a derived class from `SoCCore`:
https://github.com/litex-hub/litex-boards/blob/47faaf20d50e58affda81f611561efdc073c660d/litex_boards/targets/ulx3s.py#L82

## Step 2: make our custom SoC
Since we only want to add our `buttons` peripheral, we can actually create a derived class from `BaseSoc` and save ourselves some code rewrite.

In [1]:
# This is our SoC
from litex_boards.targets.ulx3s import BaseSoC as ulx_BaseSoC
from litex_boards.platforms.ulx3s import Platform
from litex.build.generic_platform import *

class BaseSoCf(ulx_BaseSoC):
    def __init__(self, **kwargs):
        #self.platform = Platform(device=device, revision=revision, toolchain=toolchain)
        super().__init__(**kwargs)

        # add out missing button IOs
        self.platform.add_extension([
            ("button_UP", 0, Pins("R18"), IOStandard("LVCMOS33")),
            ("button_DOWN", 0, Pins("V1"), IOStandard("LVCMOS33")),
            ("button_LEFT", 0, Pins("U1"), IOStandard("LVCMOS33")),
            ("button_RIGHT", 0, Pins("H16"), IOStandard("LVCMOS33")),
        ])
        
        # instanciate our `Button` module.
        # don't forget to add it to `self.submodules` !
        self.submodules.buttons = Buttons(self.platform)
        # we have to tell litex that our button should be CSR-mapped
        # (This can be ommitted if you updated litex today)
        self.add_csr("buttons")

Now, let's put this in a single file and see how it works

```
cd button
python ./ulx3s.py --doc --build --device LFE5U-12F
python3 -m http.server --directory build/ulx3s/doc/_build/html 8080
```
http://localhost:8080/buttons.html

## Step 3: build the software
First, let's have a look at what litex generated for us:
```
build/ulx3s/software/include/generated/
├── csr.h
├── git.h
├── mem.h
├── output_format.ld
├── regions.ld
├── sdram_phy.h
├── soc.h
└── variables.mak
```
notice that `csr.h` now `contains buttons_buttons_read`, `buttons_buttons_up_extract`, etc.

We can use these macros in our program:
```
static void button_read(void)
{
	printf("Button UP   : %d\n", buttons_buttons_up_read());
	printf("Button DOWN : %d\n", buttons_buttons_down_read());
	printf("Button LEFT : %d\n", buttons_buttons_left_read());
	printf("Button RIGHT: %d\n", buttons_buttons_right_read());
}
```

Now we can build the firmware and load it:
```
cd button/firmware
make
lxterm --kernel firmware.bin /dev/ttyUSB0
```
either reset the ULX3s or type `serialboot`command

```
litex> serialboot
Booting from serial...
Press Q or ESC to abort boot completely.
sL5DdSMmkekro
[LXTERM] Received firmware download request from the device.
[LXTERM] Uploading firmware.bin to 0x40000000 (8300 bytes)...
[LXTERM] Upload complete (9.9KB/s).
[LXTERM] Booting the device.
[LXTERM] Done.
Executing booted program at 0x40000000

--============= Liftoff! ===============--

Lab004 - CPU testing software built Mar 25 2021 15:15:04

Available commands:
help                            - this command
reboot                          - reboot CPU
led                             - led test
button                          - read button
RUNTIME>button
Button UP   : 0
Button DOWN : 0
Button LEFT : 0
Button RIGHT: 0
RUNTIME>button
Button UP   : 1
Button DOWN : 0
Button LEFT : 0
Button RIGHT: 0
RUNTIME>button
Button UP   : 1
Button DOWN : 1
Button LEFT : 0
Button RIGHT: 1
RUNTIME>
``` 

## (optional) export CSR list of generate device tree
Litex can also generate CSR lists in various forms (csv, Json, svd). You can generate them by passing the flags `--csr-csv` `--csr-json` etc. to the builder  
`python -m litex_boards.targets.ulx3s --device LFE5U-85F --build --csr-json build/csr.json`

The CSR CSV can be used by tools like `litex_server` (this tool can connect to a wishbone bridge, so you can interact directly with the wishbone bus from Pyton, on your PC).

Once the json is generated, Litex can also generate device tree for linux using command `litex_json2dts`.